In [1]:
import glob
import os
import cv2
import numpy as np
import pandas as pd

In [2]:
control_path = "labelbox_mask_combination"
wref_cover_path = "wref_mask"
eval_paths = {
    "otsu" : "otsu-corrected",
    "kmeans" : "k-means_4_clusters",
    "watershed" : "watershed"
}

labelbox_masks = glob.glob("./*/*/*/{}.png".format(control_path))

labelbox_masks

['.\\Analysis_BP\\FX10\\20220121_FX10__ras1_1-15_A_2022-01-21_09-12-35\\labelbox_mask_combination.png',
 '.\\Analysis_BP\\FX10\\20220121_FX10__ras1_1-15_B_2022-01-21_09-15-30\\labelbox_mask_combination.png',
 '.\\Analysis_BP\\FX10\\20220121_FX10__ras1_16-30_A_2022-01-21_09-20-15\\labelbox_mask_combination.png',
 '.\\Analysis_BP\\FX10\\20220121_FX10__ras1_16-30_B_2022-01-21_09-22-51\\labelbox_mask_combination.png',
 '.\\Analysis_BP\\FX10\\20220121_FX10__ras1_31-40_A_2022-01-21_10-31-47\\labelbox_mask_combination.png',
 '.\\Analysis_BP\\FX10\\20220121_FX10__ras1_31-40_B_2022-01-21_10-33-32\\labelbox_mask_combination.png',
 '.\\Analysis_BP\\FX10\\20220121_FX10__ras2_1-15_A_2022-01-21_09-25-15\\labelbox_mask_combination.png',
 '.\\Analysis_BP\\FX10\\20220121_FX10__ras2_1-15_B_2022-01-21_09-28-37\\labelbox_mask_combination.png',
 '.\\Analysis_BP\\FX10\\20220121_FX10__ras2_16-30_A_2022-01-21_09-30-52\\labelbox_mask_combination.png',
 '.\\Analysis_BP\\FX10\\20220121_FX10__ras2_16-30_B_2022-01

In [13]:
for folder in labelbox_masks:
    folder_path = folder.split("label")[0]

    for eval_mask in eval_paths:

        mask = cv2.imread("{}{}.png".format(folder_path, eval_paths[eval_mask]))
        wref = cv2.imread("{}{}.png".format(folder_path, wref_cover_path))

        shape = mask.shape
        result = np.full(shape, (0, 0, 0), dtype=np.uint8)

        mask_wref_overlay = mask & wref

        cv2.imwrite('{}{}_wref.png'.format(folder_path, eval_paths[eval_mask]), mask_wref_overlay)

        print("Processed: {}{}".format(folder_path, eval_paths[eval_mask]))
        # print('{}{}_wref.png'.format(folder_path, eval_paths[eval_mask]))

Processed: .\Analysis_BP\FX10\20220121_FX10__ras1_1-15_A_2022-01-21_09-12-35\otsu-corrected
Processed: .\Analysis_BP\FX10\20220121_FX10__ras1_1-15_A_2022-01-21_09-12-35\k-means_4_clusters
Processed: .\Analysis_BP\FX10\20220121_FX10__ras1_1-15_A_2022-01-21_09-12-35\watershed
Processed: .\Analysis_BP\FX10\20220121_FX10__ras1_1-15_B_2022-01-21_09-15-30\otsu-corrected
Processed: .\Analysis_BP\FX10\20220121_FX10__ras1_1-15_B_2022-01-21_09-15-30\k-means_4_clusters
Processed: .\Analysis_BP\FX10\20220121_FX10__ras1_1-15_B_2022-01-21_09-15-30\watershed
Processed: .\Analysis_BP\FX10\20220121_FX10__ras1_16-30_A_2022-01-21_09-20-15\otsu-corrected
Processed: .\Analysis_BP\FX10\20220121_FX10__ras1_16-30_A_2022-01-21_09-20-15\k-means_4_clusters
Processed: .\Analysis_BP\FX10\20220121_FX10__ras1_16-30_A_2022-01-21_09-20-15\watershed
Processed: .\Analysis_BP\FX10\20220121_FX10__ras1_16-30_B_2022-01-21_09-22-51\otsu-corrected
Processed: .\Analysis_BP\FX10\20220121_FX10__ras1_16-30_B_2022-01-21_09-22-51\k

In [7]:
for folder in labelbox_masks:
    folder_path = folder.split("label")[0]
    masks_to_evaluate = glob.glob(folder_path + "*_wref*")
    detectron2_mask = glob.glob(folder_path + "*detectron2*combination*")
    masks_to_evaluate.extend(detectron2_mask)

    control = cv2.imread(folder).reshape(-1)
    
    print(folder_path)

    for mask in masks_to_evaluate:
        to_be_evaluated = cv2.imread(mask).reshape(-1)

        if (len(to_be_evaluated) == len(control)):
            stats = {
            "total": control.shape[0],
            "control_fg" : 0,
            "control_bg" : 0,
            "eval_fg" : 0,
            "eval_bg" : 0,
            "correct" : 0,
            "correct_bg" : 0,
            "correct_fg" : 0,
            "false" : 0,
            "false_bg" : 0,
            "false_fg" : 0,
            }

            for c, e in zip(control, to_be_evaluated):
                if c == 0:
                    stats["control_bg"] += 1
                else:
                    stats["control_fg"] += 1

                if e == 0:
                    stats["eval_bg"] += 1
                else:
                    stats["eval_fg"] += 1

                if c == e:
                    stats["correct"] += 1
                    if c == 0:
                        stats["correct_bg"] += 1
                    else:
                        stats["correct_fg"] += 1 
                if c != e:
                    stats["false"] += 1
                    if c == 0:
                        stats["false_fg"] += 1
                    else:
                        stats["false_bg"] += 1 

            file = open("{}{}_eval.txt".format(folder_path, mask.split("\\")[-1].split(".png")[0]), "w")
            file.write("{}{}_eval.txt".format(folder_path, mask.split("\\")[-1].split(".png")[0]) + "\n\n")
            

            for stat in stats:
                file.write("{}: {}\n".format(stat, stats[stat]))
                    
            file.write("\n")
            for stat in stats:
                file.write("{} (%): {}\n".format(stat, stats[stat] * 100 / control.shape[0]))

            file.close()

            print("Evaluated: {}".format(mask.split("\\")[-1]))

.\Analysis_BP\FX10\20220121_FX10__ras1_1-15_A_2022-01-21_09-12-35\
Evaluated: k-means_4_clusters_wref.png
Evaluated: otsu-corrected_wref.png
Evaluated: watershed_wref.png
.\Analysis_BP\FX10\20220121_FX10__ras1_1-15_B_2022-01-21_09-15-30\
Evaluated: k-means_4_clusters_wref.png
Evaluated: otsu-corrected_wref.png
Evaluated: watershed_wref.png
.\Analysis_BP\FX10\20220121_FX10__ras1_16-30_A_2022-01-21_09-20-15\
Evaluated: k-means_4_clusters_wref.png
Evaluated: otsu-corrected_wref.png
Evaluated: watershed_wref.png
.\Analysis_BP\FX10\20220121_FX10__ras1_16-30_B_2022-01-21_09-22-51\
Evaluated: k-means_4_clusters_wref.png
Evaluated: otsu-corrected_wref.png
Evaluated: watershed_wref.png
.\Analysis_BP\FX10\20220121_FX10__ras1_31-40_A_2022-01-21_10-31-47\
Evaluated: k-means_4_clusters_wref.png
Evaluated: otsu-corrected_wref.png
Evaluated: watershed_wref.png
.\Analysis_BP\FX10\20220121_FX10__ras1_31-40_B_2022-01-21_10-33-32\
Evaluated: k-means_4_clusters_wref.png
Evaluated: otsu-corrected_wref.png


In [8]:
eval_files = glob.glob("./*/*/*/*.txt")
eval_files

['.\\Analysis_BP\\FX10\\20220121_FX10__ras1_1-15_A_2022-01-21_09-12-35\\k-means_4_clusters_wref_eval.txt',
 '.\\Analysis_BP\\FX10\\20220121_FX10__ras1_1-15_A_2022-01-21_09-12-35\\otsu-corrected_wref_eval.txt',
 '.\\Analysis_BP\\FX10\\20220121_FX10__ras1_1-15_A_2022-01-21_09-12-35\\watershed_wref_eval.txt',
 '.\\Analysis_BP\\FX10\\20220121_FX10__ras1_1-15_B_2022-01-21_09-15-30\\k-means_4_clusters_wref_eval.txt',
 '.\\Analysis_BP\\FX10\\20220121_FX10__ras1_1-15_B_2022-01-21_09-15-30\\otsu-corrected_wref_eval.txt',
 '.\\Analysis_BP\\FX10\\20220121_FX10__ras1_1-15_B_2022-01-21_09-15-30\\watershed_wref_eval.txt',
 '.\\Analysis_BP\\FX10\\20220121_FX10__ras1_16-30_A_2022-01-21_09-20-15\\k-means_4_clusters_wref_eval.txt',
 '.\\Analysis_BP\\FX10\\20220121_FX10__ras1_16-30_A_2022-01-21_09-20-15\\otsu-corrected_wref_eval.txt',
 '.\\Analysis_BP\\FX10\\20220121_FX10__ras1_16-30_A_2022-01-21_09-20-15\\watershed_wref_eval.txt',
 '.\\Analysis_BP\\FX10\\20220121_FX10__ras1_16-30_B_2022-01-21_09-22-51\\

In [9]:
methods = {
    "otsu": [],
    "kmeans": [],
    "watershed": [],
    "detectron": [],
}

for file in eval_files:
    if "otsu" in file:
        methods["otsu"].append(file)
    elif "means" in file:
        methods["kmeans"].append(file)
    elif "watershed" in file:
        methods["watershed"].append(file)
    elif "detectron" in file:
        methods["detectron"].append(file)

with pd.ExcelWriter("./results.xlsx") as writer:
    for method in methods:
        print(len(methods[method]))

        method_data = {}

        for i, files in enumerate(methods[method]):

            file = open(methods[method][i] ,"r")

            method_data[i] = {} 

            for idx, line in enumerate(file):
                if idx == 0:
                    method_data[i]["path"] = line.strip()
                elif 1 < idx < 13:
                    line_data = line.split(":")
                    method_data[i][line_data[0].strip()] = int(line_data[1].strip())
            
            file.close()

        df = pd.DataFrame(data=method_data).T
        df.to_excel(writer, sheet_name= method)

146
146
146
10


Purge files

In [5]:
if False:
    for file in eval_files:
        os.remove(file)
        print("Removed: {}".format(file))

Removed: .\Analysis_BP\FX10\20220121_FX10__ras1_1-15_A_2022-01-21_09-12-35\k-means_4_clusters_wref_eval.txt
Removed: .\Analysis_BP\FX10\20220121_FX10__ras1_1-15_A_2022-01-21_09-12-35\otsu-corrected_wref_eval.txt
Removed: .\Analysis_BP\FX10\20220121_FX10__ras1_1-15_A_2022-01-21_09-12-35\watershed_wref_eval.txt
Removed: .\Analysis_BP\FX10\20220121_FX10__ras1_1-15_B_2022-01-21_09-15-30\k-means_4_clusters_wref_eval.txt
Removed: .\Analysis_BP\FX10\20220121_FX10__ras1_1-15_B_2022-01-21_09-15-30\otsu-corrected_wref_eval.txt
Removed: .\Analysis_BP\FX10\20220121_FX10__ras1_1-15_B_2022-01-21_09-15-30\watershed_wref_eval.txt
Removed: .\Analysis_BP\FX10\20220121_FX10__ras1_16-30_A_2022-01-21_09-20-15\k-means_4_clusters_wref_eval.txt
Removed: .\Analysis_BP\FX10\20220121_FX10__ras1_16-30_A_2022-01-21_09-20-15\otsu-corrected_wref_eval.txt
Removed: .\Analysis_BP\FX10\20220121_FX10__ras1_16-30_A_2022-01-21_09-20-15\watershed_wref_eval.txt
Removed: .\Analysis_BP\FX10\20220121_FX10__ras1_16-30_B_2022-01-